In [1]:
# import and setup modules we'll be using in this notebook
import logging
import itertools

import numpy as np
import gensim

# logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
# logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

Using TensorFlow backend.


In [ ]:
# load cleaned email texts 
import cPickle 
f = file('/notebooks/LDA models and data/Data Frames and lists/text_term_matrix_clean.pkl', 'rb')
text_term_matrix=cPickle.load(f)

text_clean_path = file('/notebooks/LDA models and data/Data Frames and lists/text_clean.pkl', 'rb')
text_clean=cPickle.load(text_clean_path)

dictfile=file('/notebooks/LDA models and data/Data Frames and lists/Dictionary_tfidf_05V2.pkl', 'rb')
Dictionary=cPickle.load(dictfile)

In [3]:
print(Dictionary)

Dictionary(21856 unique tokens: [u'chudson', u'polytechniqu', u'suzann', u'francesco', u'ebtd']...)


In [ ]:
dictfile=file('/notebooks/LDA models and data/Data Frames and lists/Dictionary.pkl', 'rb')
Dictionary2=cPickle.load(dictfile)

In [5]:
print(Dictionary2)

Dictionary(18297 unique tokens: [u'setc', u'woodi', u'electrabel', u'yellow', u'doucett']...)


In [ ]:
# ignore words that appear in less than 20 documents or more than 10% documents
Dictionary.filter_extremes(no_below=50, no_above=0.1)
print(Dictionary)

In [ ]:
text_term_matrix0 = [Dictionary.doc2bow(text) for text in text_clean]

In [ ]:
# CHECK
# apparantly some emails got stripped of all words through our processing; thsi is not strange as emails like 
#'thnaks' are not unusual
import numpy as np
k=0
for i in range(0, len(text_term_matrix0)):
    if len(np.array(text_term_matrix0[i]).shape)==2:
        if np.array(text_term_matrix0[i]).shape[1]==2:
            k=k+1

l=0
for i in range(0, len(text_term_matrix0)):
    if len(np.array(text_term_matrix0[i]).shape)==1:
        if np.array(text_term_matrix0[i]).shape[0]==0:
            l=l+1
print l            
print len(text_term_matrix0)
print len(text_term_matrix0)-k

# hier gebleven

In [ ]:
# we first cretae new bags list by removing empty bags
text_term_matrix00=[] 
for i in range(0, len(text_term_matrix0)):
    if len(np.array(text_term_matrix0[i]).shape)==2:
        if np.array(text_term_matrix0[i]).shape[1]==2:
            text_term_matrix00.append(text_term_matrix0[i])

In [10]:
# define function for evaluation of cossim scores as folows: slpit test portion

import random
import numpy as np

def intra_inter(model, part1, part2, num_pairs=10000):
    # split each test document into two halves and compute topics for each half
    part1_mod = [model[bow] for bow in part1]
    part2_mod = [model[bow] for bow in part2]
    # print computed similarities (uses cossim)
    print("average cosine similarity between corresponding parts (higher is better):")
    print(np.mean([gensim.matutils.cossim(p1, p2) for p1, p2 in zip(part1_mod, part2_mod)]))
    intra = np.mean([gensim.matutils.cossim(p1, p2) for p1, p2 in zip(part1_mod, part2_mod)])
    
    p=len(part1)
    random_pairs = np.random.randint(0, p-1, size=(num_pairs, 2))
    print("average cosine similarity between 10,000 random parts (lower is better):")    
    print(np.mean([gensim.matutils.cossim(part1_mod[i[0]], part2_mod[i[1]]) for i in random_pairs]))
    inter = np.mean([gensim.matutils.cossim(part1_mod[i[0]], part2_mod[i[1]]) for i in random_pairs])
    
    
    return [intra, inter]

In [ ]:
import time
from collections import defaultdict
import numpy as np

grid_train = defaultdict(list)
grid_test = defaultdict(list)
cosine_score=[]

# num topics list to loop through for LDA output evaluation
parameter_list=[10, 20, 30, 50, 60, 75, 100, 125, 150]
  
# split into 50% training and 50% test sets for the permutation cossim test (see function intra_inter in prevous cell)
p_train = int(len(text_term_matrix00) *0.7)
rc =np.random.choice(len(text_term_matrix00), size=len(text_term_matrix00), replace=False, p=None)
cp_train=[text_term_matrix00[i] for i in rc[0:p_train]]

# Sample from corpus for evaluation of perplexity and cossimilarities; evaluation of full set of (even only test) documants (150k) takes too long
# Pull out the test part of corpus
cp_test_full=[text_term_matrix00[i] for i in rc[p_train:]]
# Make a smaller selection as perplexity computation take a lot of time
cp_test=random.sample(cp_test_full, 20000)

# DSplit each bag of words at random into two parts for cossim valuation
cp_test_part1=[]
cp_test_part2=[]
for i in range(len(cp_test)):
    p_test_inner = int(len(cp_test[i]) *0.5)
    rc =np.random.choice(len(cp_test[i]), size=len(cp_test[i]), replace=False, p=None)
    cp_test_part1.append([cp_test[i] for j in rc[0:p_test_inner]])
    cp_test_part2.append([cp_test[i] for j in rc[p_test_inner:]])

# Subselect random portion of train bags of owrds for peropoexity valuation
cp_train_perplexity_sample=random.sample(cp_train, 20000)

# for num_topics_value in num_topics_list:
for parameter_value in parameter_list:
    # print "starting pass for num_topic = %d" % num_topics_value
    print "starting pass for parameter_value = %.3f" % parameter_value
    start_time = time.time()

    # run model
    model = gensim.models.ldamulticore.LdaMulticore(corpus=cp_train, id2word=Dictionary, num_topics=parameter_value, chunksize=3125,\
                                     passes=1, eval_every=None, alpha=None, eta=None, decay=0.5)
    
    # show elapsed time for model
    elapsed = time.time() - start_time
    print "Elapsed time: %s" % elapsed
    
    perplex_test = model.bound(cp_test)
    print "Perplexity test: %s" % perplex_test
    grid_test[parameter_value].append(perplex_test)
    
    perplex_train = model.bound(cp_train_perplexity_sample)
    print "Perplexity train: %s" % perplex_train
    grid_train[parameter_value].append(perplex_train)
    
    per_word_perplex_test = np.exp2(-perplex_test / sum(cnt for document in cp_test for _, cnt in document))
    print "Per-word Perplexity test: %s" % per_word_perplex_test
    grid_test[parameter_value].append(per_word_perplex_test)
    
    per_word_perplex_train = np.exp2(-perplex_train / sum(cnt for document in cp_train for _, cnt in document))
    print "Per-word Perplexity train: %s" % per_word_perplex_train
    grid_test[parameter_value].append(per_word_perplex_train)
    
    
    cosine_score.append(intra_inter(model, cp_test_part1, cp_test_part2, num_pairs=20000))
# get `v1 = lsi.projection.u[dictionary.token2id['the']]; v2 = lsi.projection.u[dictionary.token2id['of']]` and print cossim(v1, v2)  

In [ ]:
text_term_matrix2 = [Dictionary2.doc2bow(text) for text in text_clean]

lda_20topics = gensim.models.ldamulticore.LdaMulticore(corpus=text_term_matrix, id2word=Dictionary_tfidf_05, num_topics=20, chunksize=3125,\
                                     passes=1, eval_every=None, alpha=None, eta=None, decay=0.5)

In [ ]:
# Visualize LDA model with an beautiful interactive tool :-))

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

# pyLDAvis.prepare(topic_term_dists, doc_topic_dists, doc_lengths, vocab, term_frequency, R=30, lambda_step=0.01,\
#                  mds=<function js_PCoA>, n_jobs=-1, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, sort_topics=True)
vis20=pyLDAvis.gensim.prepare(lda_20topics, text_term_matrix, Dictionary)
pyLDAvis.display(vis20)